In [ ]:
# !pip install imutils

In [1]:
import cv2
import imutils
import numpy as np
from sklearn.metrics import pairwise

import random

In [2]:
# For Game Emulation
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import time

In [3]:
# global variables
bg = None
prev_pos = 0

## Segmentation
Separate the foreground (hand) from the background (everything else).

### Separate main background

In [4]:
#--------------------------------------------------
# To find the running average over the background
#--------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

### Separate hand from segmented background

In [5]:
#---------------------------------------------
# To segment the region of hand in the image
#---------------------------------------------
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

### Recognize Hands using fingers

In [7]:
#--------------------------------------------------------------
# Create a feature vector from the segmented hand region
#--------------------------------------------------------------
def create_feature_vector(thresholded, segmented):
    # find the convex hull of the segmented hand region
    chull = cv2.convexHull(segmented)

    # find the most extreme points in the convex hull
    extreme_top    = tuple(chull[chull[:, :, 1].argmin()][0])
    extreme_bottom = tuple(chull[chull[:, :, 1].argmax()][0])
    extreme_left   = tuple(chull[chull[:, :, 0].argmin()][0])
    extreme_right  = tuple(chull[chull[:, :, 0].argmax()][0])

    extreme_points = np.array([
            extreme_top,
            extreme_bottom,
            extreme_left,
            extreme_right
        ])
    
    chull_mean = np.mean(extreme_points)
    chull_std = np.std(extreme_points)

    
    
    # find the center of the palm
    cX = int((extreme_left[0] + extreme_right[0]) / 2)
    cY = int((extreme_top[1] + extreme_bottom[1]) / 2)

#     palm_center = cX, cY
    
    # find the maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[extreme_left, extreme_right, extreme_top, extreme_bottom])[0]

    maximum_distance = distance[distance.argmax()]
    minimum_distance = distance[distance.argmin()]
    distances_mean = np.mean(distance)
    distances_std = np.std(distance)
    
#     print(type(distance))

    # calculate the radius of the circle with 80% of the max euclidean distance obtained
    radius = int(0.8 * maximum_distance)

    # find the circumference of the circle
    circumference = (2 * np.pi * radius)

    # take out the circular region of interest which has 
    # the palm and the fingers
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")

    
    # draw the circular ROI
    cv2.circle(circular_roi, (cX, cY), radius, 255, 1)

    # take bit-wise AND between thresholded hand using the circular ROI as the mask
    # which gives the cuts obtained using mask on the thresholded hand image
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    circular_roi_mean = np.mean(circular_roi)
    circular_roi_std = np.std(circular_roi)
    
    
    # compute the contours in the circular ROI
    (cnts, _) = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#     print (cnts)


    cnts_mean = [np.mean(c) for c in cnts]
    cnts_std = [np.std(c) for c in cnts]

    
    # initalize the finger count
    count = 0

    # loop through the contours found
    for c in cnts:
        # compute the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)

        # increment the count of fingers only if -
        # 1. The contour region is not the wrist (bottom area)
        # 2. The number of points along the contour does not exceed
        #     25% of the circumference of the circular ROI
        if ((cY + (cY * 0.25)) > (y + h)) and ((circumference * 0.25) > c.shape[0]):
            count += 1


            
#     print("chull_mean is ", type(chull_mean))
#     print("chull_std is ", type(chull_std))
#     print("maximum_distance is ", type(maximum_distance))
#     print("minimum_distance is ", type(minimum_distance))
#     print("distances_mean is ", type(distances_mean))
#     print("distances_std is ", type(distances_std))
#     print("radius is ", type(radius))
#     print("circumference is ", type(circumference))
#     print("circular_roi_mean is ", type(circular_roi_mean))
#     print("circular_roi_std is ", type(circular_roi_std))
#     print("cnts_mean is ", type(cnts_mean))
#     print("cnts_std is ", type(cnts_std))
#     print("count is ", type(count))           
            
            
    feature_vector = np.array([
        chull_mean,
        chull_std,
        maximum_distance,
        minimum_distance,
        distances_mean,
        distances_std,
        radius,
        circumference,
        circular_roi_mean,
        circular_roi_std,
        np.mean(cnts_mean),
        np.mean(cnts_std),
        count
    ], dtype=np.float32)
    return feature_vector

### Segment hand region

In [8]:
#---------------------------------------------
# To segment the region of hand in the image
#---------------------------------------------
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

## Train for Hand Open

In [11]:
recording = False
training_data_feature_vectors__hand_open = []

if __name__ == "__main__":
    # initialize accumulated weight
    accumWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)


    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # calibration indicator
    calibrated = False

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        if not grabbed:
            break
        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our weighted average model gets calibrated
        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successfull...")
        else:
            
            if recording:
                # segment the hand region
                hand = segment(gray)

                # check whether hand region is segmented
                if hand is not None:
                    # if yes, unpack the thresholded image and
                    # segmented region

                    (thresholded, segmented) = hand

    #                 print("Thresholded is: ", type(thresholded))
    #                 print("Segmented is: ", type(segmented))

#                     feature_vector = np.concatenate( (thresholded.flatten(), segmented.flatten()) )
    #                 print (hand.flatten())

    #                 print ("FV is: ", feature_vector, ", dtype is:", feature_vector.dtype, ", length is:", feature_vector.size)

                    # draw the segmented region and display the frame
                    cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                    # count the number of fingers
                    feature_vector = create_feature_vector(thresholded, segmented)
                    
                    training_data_feature_vectors__hand_open.append(feature_vector)

                
                # show the thresholded image
                cv2.imshow("Thresholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break
            
        # if the user pressed "q", then stop looping
        elif keypress == ord("r"):
            
            recording = not recording            
            
    
    # After the loop release the cap object
    camera.release()
    # Destroy all the windows
    cv2.destroyAllWindows()
    for i in range (1,5):
        cv2.waitKey(1)

[STATUS] please wait! calibrating...
[STATUS] calibration successfull...


In [13]:
print(training_data_feature_vectors__hand_open)
print([a.size for a in training_data_feature_vectors__hand_open])

[array([1.3537500e+02, 5.3079983e+01, 8.4899940e+01, 4.7634022e+01,
       6.7555183e+01, 1.6971954e+01, 6.7000000e+01, 4.2097342e+02,
       2.3720931e-01, 7.7738090e+00, 1.3346219e+02, 2.8021915e+01,
       3.0000000e+00], dtype=float32), array([1.30875000e+02, 4.24806938e+01, 5.44885292e+01, 4.71699066e+01,
       5.21281357e+01, 2.98454976e+00, 4.30000000e+01, 2.70176971e+02,
       1.72965109e-01, 6.63898993e+00, 1.21070915e+02, 1.93695793e+01,
       4.00000000e+00], dtype=float32), array([1.2987500e+02, 4.1507339e+01, 5.3150730e+01, 4.8259712e+01,
       5.1344872e+01, 1.9884719e+00, 4.2000000e+01, 2.6389377e+02,
       2.2238372e-01, 7.5271769e+00, 1.2719147e+02, 2.2211382e+01,
       4.0000000e+00], dtype=float32), array([148.625     ,  25.164148  ,  29.410883  ,  20.396078  ,
        25.216776  ,   3.3710856 ,  23.        , 144.51326   ,
         0.19767442,   7.097035  , 142.39928   ,  20.252836  ,
         3.        ], dtype=float32), array([147.875     ,  24.343569  ,  29.

## Training for Hand closed

In [15]:
recording = False
training_data_feature_vectors__hand_closed = []

if __name__ == "__main__":
    # initialize accumulated weight
    accumWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # calibration indicator
    calibrated = False

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        if not grabbed:
            break
        
        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our weighted average model gets calibrated
        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successfull...")
        else:
            
            if recording:
                # segment the hand region
                hand = segment(gray)

                # check whether hand region is segmented
                if hand is not None:
                    # if yes, unpack the thresholded image and
                    # segmented region

                    (thresholded, segmented) = hand

    #                 print("Thresholded is: ", type(thresholded))
    #                 print("Segmented is: ", type(segmented))

#                     feature_vector = np.concatenate( (thresholded.flatten(), segmented.flatten()) )
    #                 print (hand.flatten())

    #                 print ("FV is: ", feature_vector, ", dtype is:", feature_vector.dtype, ", length is:", feature_vector.size)

                    # draw the segmented region and display the frame
                    cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                    # count the number of fingers
                    feature_vector = create_feature_vector(thresholded, segmented)
                    
                    training_data_feature_vectors__hand_closed.append(feature_vector)

                
                # show the thresholded image
                cv2.imshow("Thresholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break
            
        # if the user pressed "q", then stop looping
        elif keypress == ord("r"):
            
            recording = not recording            
            
    
    # After the loop release the cap object
    camera.release()
    # Destroy all the windows
    cv2.destroyAllWindows()
    for i in range (1,5):
        cv2.waitKey(1)    

[STATUS] please wait! calibrating...
[STATUS] calibration successfull...


In [16]:
print(training_data_feature_vectors__hand_closed)
print([a.shape for a in training_data_feature_vectors__hand_closed])

[array([1.3887500e+02, 6.2232300e+01, 9.9403221e+01, 7.3600273e+01,
       8.8201080e+01, 1.0831434e+01, 7.9000000e+01, 4.9637164e+02,
       4.8924419e-01, 1.1158759e+01, 1.2474878e+02, 2.6264341e+01,
       3.0000000e+00], dtype=float32), array([1.3875000e+02, 6.1637550e+01, 9.9403221e+01, 7.2201111e+01,
       8.7354477e+01, 1.1300258e+01, 7.9000000e+01, 4.9637164e+02,
       4.6947673e-01, 1.0931430e+01, 1.2439574e+02, 2.6420586e+01,
       3.0000000e+00], dtype=float32), array([1.3875000e+02, 6.1607121e+01, 9.9403221e+01, 7.1610054e+01,
       8.7260780e+01, 1.1469805e+01, 7.9000000e+01, 4.9637164e+02,
       4.5465118e-01, 1.0757757e+01, 1.1773379e+02, 2.1321554e+01,
       3.0000000e+00], dtype=float32), array([1.39125000e+02, 6.18494911e+01, 9.86002045e+01, 7.36002731e+01,
       8.76513138e+01, 1.08395967e+01, 7.80000000e+01, 4.90088440e+02,
       4.89244193e-01, 1.11587591e+01, 1.24974335e+02, 2.62056770e+01,
       3.00000000e+00], dtype=float32), array([161.875    ,  35.25

## Create SVM Model

In [17]:
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))

## Train SVM Model

In [18]:
trainingData = np.array(training_data_feature_vectors__hand_open + training_data_feature_vectors__hand_closed)
labels = np.array(([1] * len(training_data_feature_vectors__hand_open) ) + ([2] * len(training_data_feature_vectors__hand_closed) ) )


# trainingData = np.reshape(trainingData, (trainingData.size,13))
labels = np.reshape(labels, (labels.size,1))

print (trainingData.shape)
print (labels.shape)

svm.train(np.array(trainingData), cv2.ml.ROW_SAMPLE, np.array(labels))

(312, 13)
(312, 1)


True

In [24]:
num = random.randint(0, trainingData.shape[0]-1)

print ("Training Data index #", num, " is ", trainingData[num])
test_data = np.reshape(trainingData[num], (trainingData[num].size,1))

print (test_data.T.shape)
svm.predict(test_data.T)

Training Data index # 86  is  [121.625       53.532555    83.761566    59.682495    74.02171
  10.076485    67.         420.97342      0.59302324  12.282885
 106.08213     23.899023     7.        ]
(1, 13)


(0.0, array([[2.]], dtype=float32))

In [25]:
## Initialize Selenium Browser

# https://sites.google.com/a/chromium.org/chromedriver/home
# Download ChromeDriver and extract. Then enter the fullpath here.
PATH_TO_CHROME = r"/Users/saadbazaz/Documents/FAST Studies/Digital Image Processing Lab/Project/Dino-Game-Hand-Gestures/t-rex-runner-gh-pages/chromedriver"

# The link to your Dino Game's index.html
DINO_GAME_LINK = r"/Users/saadbazaz/Documents/FAST Studies/Digital Image Processing Lab/Project/Dino-Game-Hand-Gestures/t-rex-runner-gh-pages/index.html"


browser = webdriver.Chrome(PATH_TO_CHROME)
res = browser.get('file://' + DINO_GAME_LINK)

In [26]:
## Initialize Selenium Action Chains

actions = ActionChains(browser)

## Run game (Predict hand gesture using SVM)

In [27]:

if __name__ == "__main__":
    # initialize accumulated weight
    accumWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # calibration indicator
    calibrated = False

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our weighted average model gets calibrated
        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successfull...")
        else:
            
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region

                (thresholded, segmented) = hand

#                 print("Thresholded is: ", type(thresholded))
#                 print("Segmented is: ", type(segmented))

#                     feature_vector = np.concatenate( (thresholded.flatten(), segmented.flatten()) )
#                 print (hand.flatten())

#                 print ("FV is: ", feature_vector, ", dtype is:", feature_vector.dtype, ", length is:", feature_vector.size)

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                # count the number of fingers
                feature_vector = create_feature_vector(thresholded, segmented)
                
#                 print("feature vector shape is = ", feature_vector.shape)
                
                feature_vector = np.reshape(feature_vector, (feature_vector.size,1))
                
                fingers = svm.predict(feature_vector.T)[1][0][0]

                
#         if results.right_hand_landmarks is not None:
#             pos_y = results.right_hand_landmarks.landmark[10].y

#             ## Draw threshold lines
#             y_pix = int(np.multiply(lower_thresh, image.shape[0]))
#             cv2.line(image, (0, y_pix), (image.shape[1],  y_pix), (0,255,0), thickness=2)
            
            
#             y_pix = int(np.multiply(upper_thresh, image.shape[0]))
#             cv2.line(image, (0, y_pix), (image.shape[1],  y_pix), (255,0,0), thickness=2)
    

                if fingers == 1:
                    # print ("Jump!")


                    if prev_pos == 1:
                        pass  
                    elif prev_pos == 2:
                        actions = ActionChains(browser)                    
                        actions.key_up(Keys.DOWN)
                        actions.send_keys(Keys.SPACE)
                        actions.perform() 
                    else:
                        actions = ActionChains(browser)
                        actions.send_keys(Keys.SPACE)
                        actions.perform()                 

                    prev_pos = 1

                elif fingers == 2:
                    # print ("Duck.")

                    if prev_pos == 1:
                        actions = ActionChains(browser)
                        actions.key_down(Keys.DOWN)
                        actions.perform()     
                    elif prev_pos == 2:
                        pass
                    else:
                        actions = ActionChains(browser)
                        actions.key_down(Keys.DOWN)
                        actions.perform()                     

                    prev_pos = 2


                else:
                    # print ("Keep running.")

                    if prev_pos == 2:
                        actions = ActionChains(browser)                    
                        actions.key_up(Keys.DOWN)
                        actions.perform() 

                    prev_pos = 0                   
                
                
                cv2.putText(clone, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
                # show the thresholded image
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break
    
    # After the loop release the cap object
    camera.release()
    # Destroy all the windows
    cv2.destroyAllWindows()
    for i in range (1,5):
        cv2.waitKey(1)    

[STATUS] please wait! calibrating...
[STATUS] calibration successfull...


/Users/saadbazaz/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/saadbazaz/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/saadbazaz/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/saadbazaz/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
